In [1]:
from transformers import AutoTokenizer, LEDForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("allenai/led-large-16384-arxiv")
model = LEDForConditionalGeneration.from_pretrained("allenai/led-large-16384-arxiv").to("cuda")

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn",device="cuda")

In [2]:
"""import os
import pandas as pd

def load_dataset(directory, has_split=True):
   Load dataset from directories
    data = []
    if has_split:  # IN-Abs & UK-Abs have train/test split
        for split in ["train-data", "test-data"]:
            judgement_path = os.path.join(directory, split, "judgement")
            summary_path = os.path.join(directory, split, "summary")

            for filename in os.listdir(judgement_path):
                judgement_file = os.path.join(judgement_path, filename)
                summary_file = os.path.join(summary_path, filename)

                if os.path.exists(summary_file):
                    with open(judgement_file, "r", encoding="utf-8") as f:
                        judgement_text = f.read()
                    with open(summary_file, "r", encoding="utf-8") as f:
                        summary_text = f.read()
                    data.append({"input_text": judgement_text, "summary_text": summary_text})
    else:  # IN-Ext has no train/test split
        judgement_path = os.path.join(directory, "judgement")
        summary_path = os.path.join(directory, "summary")

        for filename in os.listdir(judgement_path):
            judgement_file = os.path.join(judgement_path, filename)
            summary_file = os.path.join(summary_path, filename)

            if os.path.exists(summary_file):
                with open(judgement_file, "r", encoding="utf-8") as f:
                    judgement_text = f.read()
                with open(summary_file, "r", encoding="utf-8") as f:
                    summary_text = f.read()
                data.append({"input_text": judgement_text, "summary_text": summary_text})
    
    return pd.DataFrame(data)

# Load all datasets
datasets = {
    "IN-Abs": load_dataset("dataset/IN-Abs", has_split=True),
    "IN-Ext": load_dataset("dataset/IN-Ext", has_split=False),  # No train-test split
    "UK-Abs": load_dataset("dataset/UK-Abs", has_split=True),
}

# Combine into a single dataset
df = pd.concat(datasets.values(), ignore_index=True)

# Save as CSV
df.to_csv("legal_summaries.csv", index=False)
print("Dataset saved as CSV!")
"""

print("where")

where


In [3]:
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_pandas(pd.read_csv("legal_summaries.csv"))
dataset = dataset.train_test_split(test_size=0.1)  # 90% train, 10% test

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'summary_text'],
        num_rows: 7040
    })
    test: Dataset({
        features: ['input_text', 'summary_text'],
        num_rows: 783
    })
})


In [14]:
def preprocess_text(text):
    # Extract dates using regular expressions (as an example)
    dates = re.findall(r'\d{4}-\d{2}-\d{2}', text)
    
    # Extract legal entities (as an example)
    doc = nlp(text)  # Assuming `nlp` is an NLP model like Spacy
    legal_entities = [ent.text for ent in doc.ents if ent.label_ == "LAW"]
    
    # Extract key sections: Complaint, Response, Decision
    sections = {}
    
    if "Defendant's Response" in text:
        complaint_part = text.split("Defendant's Response")[0].strip()
        sections["complaint"] = complaint_part
    else:
        sections["complaint"] = "Complaint not found"
    
    if "Defendant's Response" in text and "Tribunal's Decision" in text:
        response_part = text.split("Defendant's Response")[1].split("Tribunal's Decision")[0].strip()
        sections["response"] = response_part
    else:
        sections["response"] = "Response not found"
    
    if "Tribunal's Decision" in text and "Supreme Court's Ruling" in text:
        decision_part = text.split("Tribunal's Decision")[1].split("Supreme Court's Ruling")[0].strip()
        sections["decision"] = decision_part
    else:
        sections["decision"] = "Decision not found"
    
    if "Supreme Court's Ruling" in text:
        appeal_part = text.split("Supreme Court's Ruling")[1].strip()
        sections["appeal"] = appeal_part
    else:
        sections["appeal"] = "Appeal not found"
    
    return {
        "text": text,
        "dates": dates,
        "legal_entities": legal_entities,
        "sections": sections,
    }

# Sample data
legal_text = """
A complaint was made by the appellant to the Railway Rates Tribunal against station, to station rates as being unreasonable on sugarcane imposed by the respondent Railway Co. by their rates circular No. 8 of 1953 with effect from October 1, 1953.
The Railway company in their answer pointed out that the rates imposed by the rates circular 8 of 1953 had long before the date of the complainant ceased to be in force and a new rate had come into operation from February 10, 1960.
Thereafter the complainant was allowed to amend the complaint.
The prayers were for a declaration that the rates charged were unreasonable, that a direction of refund of the excess collected or which may be collected after the date of the amendment of the complaint over the reasonable rates that may be fixed by the Tribunal and, lastly, for fixation of the rates.
The main contentions of the respondent were that the Tribunal had no jurisdiction to entertain the complaint as regards the reasonableness of rates prior to the institution of the complaint and that the Tribunal had no jurisdiction to grant refund.
The Indian Railways Act underwent several amendments and by the amendment in December 1957, section 41 was changed.
As a result of the change cl.
I of section 41 read thus: "Any com plaint that a railway administration (a) is contravening the provisions of section 28, or (b) is charging for the carriage of any commodity between two stations a rate which is unreasonable or is levying any other charge which is unreasonable shall hear and decide any such complaint in accordance with the provisions of this Chapter.
" 334 The Tribunal held that it had no jurisdiction to entertain or try the complaint as regards the rates and charges to prior to the institution of the complaint and it had no jurisdiction to grant any refund.
The appellants came up in appeal to the Supreme Court.
Held, that the words "is charging" in clause (b) and "is levying" in clause (c) of section 41 (1) of the Act must be construed to mean "is demanding a price at the present time for services to be rendered".
The Railway Rates Tribunal had no jurisdiction to entertain or try the complaint as regards the reasonableness or otherwise of rates and charges made prior to the institution of the complaint.
When the Tribunal had no jurisdiction to consider the reasonableness, or otherwise of any charges made prior to the institution of the complaint, it follows necessarily that it could have no occasion to order any refund.
For the question of refund could arise only after a decision that the charges made were more than what was reasonable.
Held, further, that neither expressly nor by necessary implication has the Railway Rates Tribunal been given any jurisdiction to make any order for refund.
, Southern Railway vs The Railway Rates Tribunal, A.I.R. 1955 (Madras) 676, referred to.
After a complaint is made the Tribunal shall hear and decide the complaint.
The complaint being that something is unreasonable all that the Tribunal has to decide is whether that thing is unreasonable or not.
A finding that it is unreasonable does not involve any consideration or decision of what would flow from the finding.
In otherwords, in making the complaint the complainant can ask only for a declaration that the rate or charge is unreasonable andit is only this declaratory relief which the Tribunal has been authorised to give.
There is no provision that the Tribunal can also give a consequential relief.
The only other thing which the Tribunal is authorised to do in connection with the complaint is to fix "such rate or charge as it considers reasonable.
" In the absence of any thing to indicate to the contrary it is reasonable to think that this fixation can only be prospective, that is, the Tribunal in making this order fixing the reasonable rate or, charge will mention a future date for this to come incoporation. 335
"""

# Summarize the legal text
summary = pipe(legal_text, max_length=150, min_length=50, do_sample=False)

# Print the summary
print("Summary:", summary[0]['summary_text'])
# Store preprocessed data in a DataFrame for further processing

Summary: [{'summary_text': 'The Indian Railways Act underwent several amendments and by the amendment in December 1957, section 41 was changed. The Tribunal held that it had no jurisdiction to entertain or try the complaint as regards the rates and charges to prior to the institution of the complaint. The appellants came up in appeal to the Supreme Court.'}]


In [13]:
from transformers import pipeline
import nltk
nltk.download('punkt')

# Initialize the summarizer
pipe = pipeline("summarization", model="facebook/bart-large-cnn", device="cuda")

sentences = nltk.sent_tokenize(legal_text)

# Step 2: Chunk the text into smaller segments of around 500 tokens each (approx 100 words)
chunk_size = 500  # Adjust based on token limits
chunks = [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]
chunked_texts = [' '.join(chunk) for chunk in chunks]

# Step 3: Summarize each chunk
summaries = [pipe(chunk, max_length=150, min_length=50, do_sample=False)[0]['summary_text'] for chunk in chunked_texts]

# Step 4: Combine the summaries from each chunk into a final summary
final_summary = ' '.join(summaries)
print("Final Summary:", final_summary)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KNPRO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Final Summary: A complaint was made by the appellant to the Railway Rates Tribunal against station rates as being unreasonable on sugarcane. The Tribunal held that it had no jurisdiction to entertain or try the complaint as regards the rates and charges to prior to the institution of the complaint. The appellants came up in appeal to the Supreme Court.


In [34]:
def split_into_chunks(text, tokenizer, max_length=16000, overlap=200):
    tokens = tokenizer.encode(text, truncation=False)  # Get token IDs
    chunks = []
    
    for i in range(0, len(tokens), max_length - overlap):  # Sliding window
        chunk = tokens[i : i + max_length]  # Slice the token list
        chunks.append(tokenizer.decode(chunk))  # Convert back to text
    
    return chunks

In [38]:
def summarize_text(text):
    chunks = split_into_chunks(text, tokenizer)

    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=16384).to("cuda")
        summary_ids = model.generate(**inputs, max_length=2000)  # Summarize each chunk
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summarized_chunks.append(summary)

    final_summary = " ".join(summarized_chunks)  # Merge summaries
    return final_summary


In [39]:
input_text = dataset['train'][0]['input_text']
summary = summarize_text(input_text)
print(summary)


 an appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 1 of 1960.appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 1 of 1960.appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 1 of 1960.appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 
 1 of 1960.appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 
 1 of 1960.appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 
 1 of 1960.appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 
 1 of 1960.appeal by special leave from the judgment and order dated,, Railway Rates Tribunal, Madras in Complaint no. 
 1 of 1960.appeal by special leave from the judgment 

A complaint was made by the appellant to the Railway Rates Tribunal against station, to station rates as being unreasonable on sugarcane imposed by the respondent Railway Co. by their rates circular No. 8 of 1953 with effect from October 1, 1953.
The Railway company in their answer pointed out that the rates imposed by the rates circular 8 of 1953 had long before the date of the complainant ceased to be in force and a new rate had come into operation from February 10, 1960.
Thereafter the complainant was allowed to amend the complaint.
The prayers were for a declaration that the rates charged were unreasonable, that a direction of refund of the excess collected or which may be collected after the date of the amendment of the complaint over the reasonable rates that may be fixed by the Tribunal and, lastly, for fixation of the rates.
The main contentions of the respondent were that the Tribunal had no jurisdiction to entertain the complaint as regards the reasonableness of rates prior t